In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\output_data\cfb\cfb_conf_aff.csv')


In [74]:
tmc_2014 = df[df["Season"] == 2014][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2015 = df[df["Season"] == 2015][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2016 = df[df["Season"] == 2016][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2017 = df[df["Season"] == 2017][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2018 = df[df["Season"] == 2018][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2019 = df[df["Season"] == 2019][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2020 = df[df["Season"] == 2020][df["Conference"].isna()]["Team"].unique().tolist()
tmc_2020

C:\Users\lyons\AppData\Local\Temp\ipykernel_4532\4137911192.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmc_2014 = df[df["Season"] == 2014][df["Conference"].isna()]["Team"].unique().tolist()
C:\Users\lyons\AppData\Local\Temp\ipykernel_4532\4137911192.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmc_2015 = df[df["Season"] == 2015][df["Conference"].isna()]["Team"].unique().tolist()
C:\Users\lyons\AppData\Local\Temp\ipykernel_4532\4137911192.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmc_2016 = df[df["Season"] == 2016][df["Conference"].isna()]["Team"].unique().tolist()
C:\Users\lyons\AppData\Local\Temp\ipykernel_4532\4137911192.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmc_2017 = df[df["Season"] == 2017][df["Conference"].isna()]["Team"].unique().tolist()
C:\Users\lyons\AppData\Local\Temp\ipykernel_4532\4137911192.py:5: UserWarnin

['southern-methodist-mustangs',
 'tcu-horned-frogs',
 'texas-longhorns',
 'north-dakota-state-bison',
 'arkansas-razorbacks']

In [75]:
mtm_2014 = {
    'michigan-wolverines' : 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'arizona-wildcats' : 'pac-12',
    'north-dakota-state-bison' : 'mvfc',
    'utah-utes' : 'pac-12',
    'virginia-cavaliers' : 'acc'
}

mtm_2015 = {
    'arizona-wildcats' : 'pac-12',
    'michigan-wolverines' : 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'north-carolina-tar-heels' : 'acc'
}

mtm_2016 = {
    'michigan-wolverines' : 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'north-dakota-state-bison' : 'mvfc',
    'southern-methodist-mustangs' : 'american' 
}

mtm_2017 = {
    'michigan-wolverines': 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'iowa-hawkeyes' : 'big-ten',
    'virginia-cavaliers' : 'acc'
}

mtm_2018 = {
    'florida-gators' : 'sec',
    'michigan-wolverines' : 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'virginia-cavaliers' : 'acc'
}

mtm_2019 = {
    'arizona-wildcats' : 'pac-12',
    'michigan-wolverines' : 'big-ten',
    'tcu-horned-frogs' : 'big-12',
    'texas-longhorns' : 'big-12',
    'utah-utes' : 'pac-12',
    'north-dakota-state-bison' : 'mvfc',
    'southern-methodist-mustangs' : 'american',
    'louisiana-lafayette-ragin-cajuns' : 'sbc'
}

mtm_2020 = {
    'southern-methodist-mustangs' : '',
    'tcu-horned-frogs' : '',
    'texas-longhorns' : '',
    'north-dakota-state-bison' : '',
    'arkansas-razorbacks' : ''
}